## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
from sklearn.metrics import accuracy_score, roc_curve, auc

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Separate target variable and features
y = model_data['Angaus']  # Target variable (presence/absence of eel species)
X = model_data.drop('Angaus', axis=1)  # Features

# Apply label encoding to the 'Method' column
label_encoder = LabelEncoder()
X['Method'] = label_encoder.fit_transform(X['Method'])

# Split data into training and validation sets for early stopping
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=808)


### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [4]:
# Create XGB model classifier
xgb_model = xgb.XGBClassifier(n_estimators = 1000,
                              learning_rate = 0.1,
                              eval_metric = "logloss", 
                              early_stopping_rounds = 50,
                              random_state = 808,
                             n_jobs = -1) # A tip from Nicole for trying to optimize processing

# Fit model
xgb_model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 0)

# Print the best number of trees
best_trees = xgb_model.best_iteration
print(f"Best number of trees: {best_trees}")

Best number of trees: 29


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [5]:
# Define hyperparameter distributions
param_dist = {
    "learning_rate":uniform(0.01, 0.31)
}


# Put a model on the best trees
best_tree_model = xgb.XGBClassifier(n_estimators = best_trees,
                                   random_state = 808)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(best_tree_model, 
                                   param_dist, 
                                   n_iter = 20,
                                   cv = 5, 
                                   random_state = 808,
                                  n_jobs = -1)

# Run random search
random_search.fit(X_train, y_train)

# Print the best learning rate
best_learning_rate = random_search.best_params_['learning_rate']
print(f"Best learning rate: {best_learning_rate}")

Best learning rate: 0.10161649960684291


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [11]:
# Define XGBoost model
model_again = xgb.XGBClassifier(
    n_estimators = best_trees,
    learning_rate  = best_learning_rate,
    random_state = 808,
    #verbose=0,
    n_jobs = -1)

# Define parameter dictionary
param_dist = {
    'max_depth': randint(3, 11),
    'min_child_weight': randint(1, 11),
    'gamma': uniform(0.05, 0.05),
}

# Find best combination of parameteres
random_search = RandomizedSearchCV(
    model_again,
    param_dist,
    n_iter = 20, 
    random_state = 808,
    cv = 5,
    n_jobs = -1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Print the best parameters...
best_params_sto = random_search.best_params_
print(f"Best stochastic params: {best_params_sto}")

Best stochastic params: {'gamma': 0.05534152228259613, 'max_depth': 10, 'min_child_weight': 5}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [12]:
# Define XGBoost model...again
model_stochastic = xgb.XGBClassifier(
    n_estimators = best_trees,
    learning_rate  = best_learning_rate,
    **best_params_sto, 
   eval_metric = "logloss", 
    random_state = 808,
    n_jobs = -1)

# Define dictionary
param_dist = {
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5)
}

# Find best combination of parameters
random_search = RandomizedSearchCV(
    model_stochastic,
    param_dist,
    n_iter = 20,
    random_state = 808,
    n_jobs = -1)

random_search.fit(X_train, y_train)

print(random_search.best_params_)

truly_best_params_sto = random_search.best_params_
print(f"Best stochastic params: {truly_best_params_sto}")

{'colsample_bytree': 0.7435738316840477, 'subsample': 0.8699203563223885}
Best stochastic params: {'colsample_bytree': 0.7435738316840477, 'subsample': 0.8699203563223885}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [13]:
# Prepare the data, but now on the eval 
y_test = eval_data['Angaus_obs']
X_test = eval_data.drop('Angaus_obs', axis = 1)

# Label encoder stuff
le = LabelEncoder()

# Encode the "Method" column, again, but on other df
X_test["Method"] = le.fit_transform(X_test["Method"])

# Define XGBoost model...once again!
final_model = xgb.XGBClassifier(n_estimators = best_trees,
                          learning_rate = best_learning_rate,
                          **best_params_sto,
                          **truly_best_params_sto,
                          eval_metric = "logloss",
                          random_state = 808, 
                          n_jobs = -1)

# Fit the model
final_model.fit(X, y)

# Predict!!!!!!
y_pred = final_model.predict(X_test)

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [17]:
# Calculate probability
final_model_probs = final_model.predict_proba(X_test)[:, 1]

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, final_model_probs)

# Figure out auc
auc = roc_auc_score(y_test, final_model_probs)

# Print it!
print(f"AUC: {auc}")

# Find out feature importance....and make a dataframe out of it
feature_importance = final_model.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values('Importance', ascending = False)

#Print the dataframe!
print(importance_df)

AUC: 0.8716677367957956
       Feature  Importance
0      SegSumT    0.211059
10      Method    0.141461
4   DSMaxSlope    0.140930
8     USNative    0.110642
6   USRainDays    0.088408
3       DSDist    0.057238
1     SegTSeas    0.057209
7      USSlope    0.052555
5       USAvgT    0.051734
11      LocSed    0.046673
2   SegLowFlow    0.042089
9        DSDam    0.000000


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

*The XGB model is more accurate that that of Elith et al, according to our AUC score. However, it was a beast to make and run! So, there are likely more computationally efficient models, that require less iterating, which we could compare in addition to predictive performance.*